# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [81]:
# Write your code below.
%load_ext dotenv
%dotenv


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [82]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [83]:
import os
from glob import glob
import pandas as pd

# Load PRICE_DATA environment variable and find parquet files
PRICE_DATA_DIR = os.getenv("PRICE_DATA")
PRICE_DATA_FILES = glob(os.path.join(PRICE_DATA_DIR, "**/*.parquet"), recursive=True)

# Read parquet files into Dask DataFrame
dd_prices = dd.read_parquet(PRICE_DATA_FILES)

# to verify the path
# print("PRICE_DATA path is constructed as:", PRICE_DATA_DIR)

# convert to pandas DataFrame for checking
# pandas_df = dd_prices.compute() 

# to see the first few rows to make sure it works as expected
# print(pandas_df.head())

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [84]:
# Write your code below.
def calculate_returns_and_ranges(df):
    df = df.assign(close_lag_1=lambda x: x['Close'].shift(1))
    df = df.assign(adj_close_lag_1=lambda x: x['Adj Close'].shift(1))
    df = df.assign(returns=lambda x: x['Close'] / x['close_lag_1'] - 1)
    df = df.assign(adj_returns=lambda x: x['Adj Close'] / x['adj_close_lag_1'] - 1)
    df = df.assign(hi_lo_range=lambda x: x['High'] - x['Low'])
    return df

# Read all files as one dask DataFrame
dd_px = dd.read_parquet(PRICE_DATA_FILES).set_index("ticker")

# Group by ticker and apply calculations and assign the result to `dd_feat`.
dd_feat = dd_px.groupby('ticker', group_keys=False).apply(calculate_returns_and_ranges)


C:\Users\mdmee\AppData\Local\Temp\ipykernel_105280\3826581830.py:14: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = dd_px.groupby('ticker', group_keys=False).apply(calculate_returns_and_ranges)


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [85]:
# Write your code below.
# Convert to pandas DataFrame
pdf_feat = dd_feat.compute()

# Calculate the moving average of returns with a window of 10 days
pdf_feat['returns_moving_avg'] = pdf_feat.groupby('ticker')['returns'].rolling(window=10).mean().reset_index(level=0, drop=True)

# Display the first few lines of the pandas DataFrame
print(pdf_feat.head())


             Date   Open   High    Low  Close  Adj Close    Volume    source  \
ticker                                                                         
AAOI   2013-09-26  10.00  10.09   9.37   9.96       9.96  946000.0  AAOI.csv   
AAOI   2013-09-27  10.44  10.44  10.00  10.10      10.10  253300.0  AAOI.csv   
AAOI   2013-09-30  10.00  10.18   9.71  10.00      10.00   84900.0  AAOI.csv   
AAOI   2013-10-01   9.95  10.02   9.92  10.00      10.00   74500.0  AAOI.csv   
AAOI   2013-10-02   9.99  10.00   9.89   9.97       9.97   94000.0  AAOI.csv   

        Year  close_lag_1  adj_close_lag_1   returns  adj_returns  \
ticker                                                              
AAOI    2013          NaN              NaN       NaN          NaN   
AAOI    2013         9.96             9.96  0.014056     0.014056   
AAOI    2013        10.10            10.10 -0.009901    -0.009901   
AAOI    2013        10.00            10.00  0.000000     0.000000   
AAOI    2013        10.00

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

No, it was not strictly necessary to convert to pandas to calculate the moving average.

Yes, I think it would have been better to do it in Dask. Because, Dask is often better for handling larger-than-memory datasets efficiently, distributing computations across multiple cores or clusters, and avoiding loading the entire dataset into memory. This could have saved us some time as we calculated the moving average using the rolling feature.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [X] Created a branch with the correct naming convention.
- [X] Ensured that the repository is public.
- [X] Reviewed the PR description guidelines and adhered to them.
- [X] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.